# Goals

# Libraries

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.mstats import zscore
from factor_analyzer import FactorAnalyzer
from factor_analyzer import confirmatory_factor_analyzer
import pingouin as pg
from linearmodels import PanelOLS
from linearmodels.panel import compare as compare_models
import statsmodels.api as sm
from io import StringIO

/Users/otto/opt/miniconda3/envs/19cStates/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Parameters & Directories

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.figsize"] = (6,6)
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams["font.style"] = "normal"
plt.rcParams["axes.labelcolor"] = "gray"
plt.rcParams["text.color"] = "grey"
pd.set_option('mode.chained_assignment', None)

In [3]:
DIR = os.path.dirname(os.getcwd()) + "/"
DATA = DIR + 'data/'

# Read in Data

In [4]:
states_df = pd.read_csv(DATA + "interim/" + "int_19cStates.csv")
states_df.shape

(2094, 81)

In [5]:
# states_df.groupby('state').size()

In [6]:
# states_df = states_df[states_df['state']!= 'Italy']
# states_df.groupby('state').size()

Average Panel Size

In [7]:
round(states_df.groupby('state').size().mean())

105

In [8]:
# for col in sorted(states_df.columns):
#     print(col)

# Set Globals

In [9]:
# Development of states
DEV_STATE = ['sy_gdp',
             'sy_urban',
             'sy_lifeex',
             'sy_suffrage',
            ]

# Conflict
CONFLICT = ['sy_war',
            'sy_riots'
           ]


# Nationalization of states 
NAT_STATE = [
             'sy_anthem',
             'sy_xtr_enrll',
             'sy_citizenship',
             'sy_nat_bank',
             'sy_dom_autonomy',
             'sy_int_autonomy',
             'sy_flag'
            ]


# Scientization of states
SCI_STATE = ['sy_stats_agency',
             'sy_yrbk_cov',
             'sy_yrbk_pub',
             'sy_unis',
             'sy_acad_free', 
             'sy_census_cum', 
            ]

# Degree of nationalization of states, worldwide
NAT_WORLD = [
             'wy_anthem',
             'wy_xtr_enrll',
             'wy_citizenship',
             'wy_nat_bank',
             'wy_dom_autonomy',
             'wy_int_autonomy', 
             'wy_flag'    
            ]

# Degree of scientization of states, worldwide
SCI_WORLD = ['wy_stats_agency',
             'wy_yrbk_cov',
             'wy_yrbk_pub',
             'wy_unis',
             'wy_acad_free',
             'wy_census_ever',
             'wy_confs_sci_ref',
             'wy_stats_journals',
             'wy_soc_journals',
             'wy_societies', 'sy_confs_sci_ref',
            ]

# Sci and nat globals
GLOBALS = {"sy_dev_state": DEV_STATE,
           "sy_nat_state": NAT_STATE,
           "sy_sci_state": SCI_STATE,
           "sy_state_mdl": NAT_STATE+SCI_STATE,
           "wy_nat_world": NAT_WORLD,
           "wy_sci_world": SCI_WORLD,
           "wy_state_mdl": NAT_WORLD+SCI_WORLD,
          }

ALL = DEV_STATE + NAT_STATE + SCI_STATE + NAT_WORLD + SCI_WORLD + CONFLICT

In [10]:
GLOBALS['sy_state_mdl']

['sy_anthem',
 'sy_xtr_enrll',
 'sy_citizenship',
 'sy_nat_bank',
 'sy_dom_autonomy',
 'sy_int_autonomy',
 'sy_flag',
 'sy_stats_agency',
 'sy_yrbk_cov',
 'sy_yrbk_pub',
 'sy_unis',
 'sy_acad_free',
 'sy_census_cum']

# Reliability & Factor Analysis

Inspect alphas

In [11]:
states_df = states_df.dropna(how='any', subset=ALL)
states_df.shape
states_df.to_csv(DATA+'interim/'+"test.csv")

In [12]:
alphas = []
for construct, indicators in GLOBALS.items():
    alpha = pg.cronbach_alpha(zscore(states_df[indicators]))[0]
    alphas.append((construct, alpha))
cronbachs = pd.DataFrame(alphas, columns=['construct', 'Cronbach\'s alpha']).round(3)
cronbachs

,construct,Cronbach's alpha
0,sy_dev_state,0.718
1,sy_nat_state,0.760
2,sy_sci_state,0.724
3,sy_state_mdl,0.800
4,wy_nat_world,0.974
5,wy_sci_world,0.968
6,wy_state_mdl,0.982


Inspect eigenvalues

In [13]:
factor = FactorAnalyzer()
eigens = []
for construct, indicators in GLOBALS.items():
    z = factor.fit(zscore(states_df[indicators]))
    vs = [construct]
    for v in z.get_eigenvalues()[1]:
        vs.append(v)
    eigens.append(vs)
eigens = pd.DataFrame(eigens)
eigens.iloc[:,:6].round(3)

,0,1,2,3,4,5
0,sy_dev_state,1.666,0.319,0.099,-0.272,NaN
1,sy_nat_state,2.782,0.752,0.698,0.117,-0.014
2,sy_sci_state,2.673,0.998,0.688,0.098,-0.015
3,sy_state_mdl,3.714,2.011,1.145,0.548,0.217
4,wy_nat_world,5.892,0.562,-0.012,-0.056,-0.098
5,wy_sci_world,8.361,0.604,0.083,0.005,-0.047
6,wy_state_mdl,14.000,1.701,0.468,0.289,0.243


Predict 

In [14]:
DVs = []
for construct, indicators in GLOBALS.items():
    factor.fit(zscore(states_df[indicators]));
    states_df[construct] = zscore(factor.transform(pd.DataFrame(zscore(states_df[indicators]))).T[0])

# Descriptive Statistics

## Univariate

In [15]:
states_df[ALL+list(GLOBALS.keys())].agg(['count', 'mean', 'std', 'min', 'max']).T.round(3)

,count,mean,std,min,max
sy_gdp,1868.0,3.390,1.772,0.923,9.518
sy_urban,1868.0,2806.459,4171.274,14.364,25543.998
sy_lifeex,1868.0,40.746,6.283,8.110,59.100
sy_suffrage,1868.0,24.055,21.423,0.000,100.000
sy_anthem,1868.0,0.544,0.498,0.000,1.000
sy_xtr_enrll,1868.0,58.112,29.040,0.276,99.997
sy_citizenship,1868.0,0.861,0.346,0.000,1.000
sy_nat_bank,1868.0,0.531,0.499,0.000,1.000
sy_dom_autonomy,1868.0,1.036,1.005,-2.437,2.072
sy_int_autonomy,1868.0,0.819,1.364,-2.353,1.971


## Bivariate

In [16]:
pd.DataFrame(np.tril(states_df[DEV_STATE+list(GLOBALS.keys())].corr().round(3).to_numpy()), 
             columns=DEV_STATE+list(GLOBALS.keys()),
             index=DEV_STATE+list(GLOBALS.keys()))

,sy_gdp,sy_urban,sy_lifeex,sy_suffrage,sy_dev_state,sy_nat_state,sy_sci_state,sy_state_mdl,wy_nat_world,wy_sci_world,wy_state_mdl
sy_gdp,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
sy_urban,0.442,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
sy_lifeex,0.556,0.159,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
sy_suffrage,0.584,0.195,0.395,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
sy_dev_state,0.953,0.311,0.759,0.657,1.000,0.000,0.000,0.000,0.000,0.000,0.0
sy_nat_state,0.325,0.402,0.181,0.222,0.288,1.000,0.000,0.000,0.000,0.000,0.0
sy_sci_state,0.513,0.356,0.483,0.235,0.531,0.216,1.000,0.000,0.000,0.000,0.0
sy_state_mdl,0.174,0.359,0.101,0.130,0.144,0.942,0.211,1.000,0.000,0.000,0.0
wy_nat_world,0.615,0.286,0.509,0.582,0.660,0.183,0.650,0.187,1.000,0.000,0.0
wy_sci_world,0.550,0.248,0.417,0.536,0.582,0.158,0.577,0.168,0.947,1.000,0.0


# Standardize Data

In [17]:
for col in ALL+list(GLOBALS.keys()): 
    states_df["sd_"+col] = zscore(states_df[col])

In [18]:
states_df['sd_sy_model'] = (states_df['sd_sy_nat_state'] + states_df['sd_sy_sci_state']) / 2
states_df['sd_wy_model'] = (states_df['sd_wy_nat_world'] + states_df['sd_wy_sci_world']) / 2


In [19]:
states_df['sd_sci:nat'] = states_df['sd_sy_nat_state'] * states_df['sd_sy_sci_state']

In [20]:
states_df.shape

(1868, 135)

# Fit Models

## Helper functions 
Since we'll be running 6 models x 3 dvs x 5 robustness checks, or ~90 models, we'll create some helper functions to reduce repeated code.

In [21]:
def nest_models(m0, mn):
    '''
    takes a baseline model;
    returns taxonomy of nested models
    with m3 modifiable
    '''
    m1 = m0 + ['l_sy_war', 'l_sy_riots']
    ms = [m0, m1]
    ms.append(ms[1] + mn)
    ms.append(ms[2] + ['l_wy_nat_world'])
    ms.append(ms[2] + ['l_wy_sci_world'])
    ms.append(ms[2] + ['l_wy_model'])
    return ms

In [22]:
def gen_model_df(df, models, lag=3):
    '''
    takes a standardized df, a set of regressors, and a lag 
    returns a lagged df with no missingness due to lag
    '''
    
    # New columns names
    COLS = [col for col in states_df.columns if 'sd_' in col]
    LALL = [col.replace('sd_', 'l_') for col in COLS]
    cmap = dict(zip(COLS, LALL))
    
    # Lag standardized cols
    model_df = df.join(df.sort_values(by=['state','year'])
                       .groupby('state')[COLS]
                       .shift(lag)
                       .rename(columns=cmap), on=df.index)
    
    # Set make state categorical to make it panel var
    state_ids = pd.Categorical(df.state)
    model_df = model_df.set_index(["state","year"])
    model_df['state'] = state_ids
    
    # Drop missing
    model_df = model_df.dropna(how='any', subset=[m for m in models][0])
    return model_df

In [23]:
def run_models(dv, models, df):
    '''
    takes a dv, a sequence of models to fit, and a df
    returns results of fitting taxonomy of FE models
    '''
    results = {}
    i = 0
    for model in models: 
        i += 1
        regressors = sm.add_constant(df[model])
        result = PanelOLS(df[dv],
                           regressors,
                           entity_effects=True,
                           check_rank=False).fit(cov_type="clustered",
                                                 cluster_entity=True)
        
        if f'Model {i}' not in results: 
            results[f'Model {i}'] = result
    return results

In [24]:
def write_table(path, results, table_n):
    '''
    takes a path, PanelOLS results obj, and table number
    transforms PanelOLS results object into a pandas df
    writes an excel of results at specified path
    '''
    results_obj = compare_models(results, precision='std_errors', stars=True)
    results_csv = results_obj.summary.as_csv()
    results_pth = StringIO(results_csv)
    results_df = pd.read_csv(results_pth, skiprows=1, skipfooter=4, engine='python')
    results_df.drop(results_df.index[[10]]).to_excel(path+f'table_{table_n}.xlsx', 
                        index=False, 
                        engine='xlsxwriter', 
                        sheet_name=f'table_{table_n}')

## Run models

### Tables 2, 3, 4 & Appendix B
Main report results and robustness checks for lag period

In [25]:
dvs = ['sy_nat_state', 'sy_sci_state', 'sd_sy_model']
m0 = ['l_sy_suffrage', 'l_sy_lifeex', 'l_sy_urban','l_sy_gdp']
lags = [5,3,7]
tbl = 1
tblb = 0
for lag in lags:
    for dv in dvs:
        
        # nest models & gen lagged df
        # nested m3 depends on dv
        if 'model' not in dv: 
            m3 = ['l_sy_nat_state', 'l_sy_sci_state']
        else:
            m3 = ['l_sy_model']
        ms = nest_models(m0, m3)
        df = gen_model_df(df=states_df, lag=lag, models=ms)
        
        # Run model & output results
        results = run_models(dv, ms, df)
    
        # robustness checks
        if lag != 5:
            tblb += 1
            write_table(path = DIR+"/models/",
                       results=results,
                       table_n=f'b{tblb}-{lag}')
            
        # main results
        else: 
            tbl += 1
            write_table(path = DIR+"/models/", 
                       results=results,
                       table_n=tbl)

### Appendix C
Modeling development as an integrated historical phenom

In [26]:
dvs = ['sy_nat_state', 'sy_sci_state', 'sy_state_mdl']
m0 = ['l_sy_dev_state']
j = 0
for dv in dvs: 
    j += 1
    if 'mdl' not in dv: 
        m3 = ['l_sy_nat_state', 'l_sy_sci_state']
    else:
        m3 = ['l_sy_state_mdl']
    ms = nest_models(m0, m3)
    df = gen_model_df(df=states_df, lag=lag, models=ms)
    results = run_models(dv, ms, df)
    write_table(path = DIR+"/models/",
               results=results, 
               table_n=f'c{j}')

### Appendix D
Testing whether nationalization depends on scientizaiton

In [28]:
dvs = ['sy_nat_state', 'sy_sci_state']
m0 = ['l_sy_suffrage', 'l_sy_lifeex', 'l_sy_urban','l_sy_gdp']

j = 0
for dv in dvs: 
    j += 1
    m3 = ['l_sy_nat_state', 'l_sy_sci_state', 'l_sci:nat']
    ms = nest_models(m0, m3)
    df = gen_model_df(df=states_df, lag=lag, models=ms)
    results = run_models(dv, ms, df)
    write_table(path = DIR+"/models/",
               results=results, 
               table_n=f'd{j}')

# Output processed data
This will be used for visualizations

In [29]:
states_df.to_csv(DATA+'processed/pro_19cStates.csv', index=False)

In [30]:
states_df.shape

(1868, 135)